In [50]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.request as urllib2
import re
import csv
import pandas as pd
import os
import numpy as np

bank = 'PNC Bank'
listId = 1
jobNo = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'freq100.csv'
    elif listId == 2:
        return 'textRank.csv'
    else:
        return 'tfIdf.csv'

def scrape_site(Url):
    scrape_page(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        print("page - " + nextPageUrl)
        scrape_page(nextPageUrl)

def scrape_page(page):
    response = urlopen(page)
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        links = bs_obj.findAll('div',{'class': 'cmp-section cmp-with-border'})
        for page in links:
            linkTags = page.find_all('a', attrs={'href': re.compile("/cmp/_")})
            for tag in linkTags:
                link = tag.get('href')
                postingUrl = BASE + link
                pageDict = scrape(postingUrl)
                if (type(pageDict) is dict):
                        data = pd.DataFrame([pageDict])
                        # if file does not exist write header 
                        if not os.path.isfile('pncData.csv'):
                            data.to_csv('pncData.csv', header='column_names')
                        else: 
                            # else it exists so append without writing the header
                            data.to_csv('pncData.csv', mode='a', header=False)
                else:
                    print("exception?")


def scrape(postingUrl):
    print("posting - " + postingUrl)
    global bank
    global listId
    global jobNo
    
    
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]
    values = [jobNo, bank, listId, postingUrl, listId]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open(keywordsFileName(listId), 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC Bank':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
    except:
        return ("An exception occurred")
    
    jobNo = jobNo + 1

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 



                
if __name__ == '__main__':
    scrape_site(LISTING_URL_PREFIX + LISTING_URL_POSTFIX)

posting - https://www.indeed.com//cmp/_/job?jk=f5df4ed098b9df5e&tk=1d377raj40mtv000
posting - https://www.indeed.com//cmp/_/job?jk=b9f01fc8684726f8&tk=1d377raj40mtv000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXac_m9-fTbJh2bXSMOtpuQF9psoJiIjNq08-zXrPCp9FM-x_27b3HnDH3kNs38fcgo7XTM99d0lb97QbOjnG2zQZT1gtafDX3-7_8SZg4pqgnCvr4CAvSgfxF-_s72KkAPmH99XhhtskH7LCpn1caujxo31yIyB4-U-34bK_P_2LMY7-qHtymM0oVWYUyRDTTq8aYjyxU_TYFWBMDt_K8fp8csQfz89zj2NE-5shAU9t6G0BrUyjmvFYwyR4J6Yhcz4Ndn1creSg1mkRgS56TXtbbznMdJqO0GV1I1okrA0MnvzccW8-AYpad_vw1MDTNzmn5fuX6MtWji7TxDr_OjzeaqMGJJXHGKJGCnWO30_b7CxZXIIwvw3blg3HfAGDyP17cKH4b6f8tiZnuT2JOMK5lLa2W8eeFHvLmVhOEGenGOuSshKs0-HR_j7xZtIUlsHHpSlets0Xv59ggst9d35A3BrIm80pa_OuSvGJaxKUaSbrfhmYr82wiCloeLLsvPs
posting - https://www.indeed.com//cmp/_/job?jk=26f29258e6500f1b&tk=1d377raj40mtv000
posting - https://www.indeed.com//cmp/_/job?jk=b1d7fe96e8c83f71&tk=1d377raj40mtv000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKi

posting - https://www.indeed.com//cmp/_/job?jk=58ec1f7ac23f0fff&tk=1d377s0g5akcg800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaeQ4p_OVg-4taKKEhUoN5WE4GY4Rk9jAafnR0G1yfFbc9rabjfyJi185cskYNliSAICf2Ym6tRUH4TH1m-nrecDIzXtOVsvoKVtoqytaKSGDYAjOJ0LetHw7PA-3quxC0FNWak2L2IHl27UlL7KnU8QNmIM8deWXIQtWZgPCafNa1WkQAPSZchGX3ps4M5VN2OITBSbwl26xQkRi3x5awbba9GES8WWRBVmKonFDgSWqgmJTHcr2RTMWV0N6Gz-ZiXaO5_qdiGyBFKrgbjlTb4WJkm0kYnLWCidUCbmRjdkrC_nLTHR_beoogtEM1b5Firmh1zjTGDObEWSmdywXS8Zc-V_1UGPaqmeEc6V5RMTMhi67bTFs9Kj_YFduq53z65eyB1iJONZZjgC07TA64jv3nvKcpw8G-NtIWomWj3gZ6NIf1KSL7s6NQEpiOnDWlTCUKqNJoq1MpUo6F7tFEF5WFIa9ILJxDxVGgSz9H9Q9MPRRByWJPeM34z2YsIKxU4
posting - https://www.indeed.com//cmp/_/job?jk=18b31206fcb52d2d&tk=1d377s0g5akcg800
posting - https://www.indeed.com//cmp/_/job?jk=593da1a235fba740&tk=1d377s0g5akcg800
posting - https://www.indeed.com//cmp/_/job?jk=5f0e0ffe66f32922&tk=1d377s0g5akcg800
page - https://www.indeed.com/cmp/Pnc-Fin

page - https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start=60&clearPrefilter=1#cmp-menu-container
posting - https://www.indeed.com//cmp/_/job?jk=e2d62d7a0d4aa89d&tk=1d37882jq0mct000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6vxZlsYClqA2a8pSwxaDhLq3MteitQTqQSkOZzvStesEY7rGN9NgAjEZhP5ccHbgZaGto_5jaQNxWLcGZB52zLf92QdDX5p7OKmCgNvh6k9zwrtXPC_XUHh6moZD2ROyNfiqm1rzpxKVCvxYYj7RXaRgJdZ_FHKZHdGw-UwdxIRZctActMpYjc95a-hMvkujH4RG1sN2OIt8QQ33BPD0sd8x9AMzMsFcUCmeEOG9YqOL0Z7yBHUFB4ysdHatiU-630jHRTIOdJKkO_zKNqk4A_hCNSUs8tV-k-cepcOEkGN-RC2_ppB5jsZwY7o0SB7vAwXi0NZjWLaBkMoCqqwkH-jEdjiEPlHwI-E7CQcxK1j4s4YpzkXkYSynqe8vuB3JboftLkLpXS4N4AauTsH7cvAP_y9IaOWglhPYB_F_neypRLjBZQA4iG3rG_P10xew2GzdkFO4NMPeVpKsrkHY0G1opMK6xEehnDp6Ymb1GE2NiI4DFyppTb_8mAya9M9fM4-K0ArAmL9zzqS2ZbcpnmM
posting - https://www.indeed.com//cmp/_/job?jk=dc835cbd410607ac&tk=1d37882jq0mct000
posting - https://www.indeed.com//cmp/_/job?jk=c212c1da742b9b63&tk=1d37882jq0mct000
posting - ht

posting - https://www.indeed.com//cmp/_/job?jk=2aee691d0b3c6ad8&tk=1d3788pad0mct000
page - https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start=84&clearPrefilter=1#cmp-menu-container
posting - https://www.indeed.com//cmp/_/job?jk=6e0e6a323523e618&tk=1d3789e6b0mct000
posting - https://www.indeed.com//cmp/_/job?jk=47910f4cf325fbed&tk=1d3789e6b0mct000
posting - https://www.indeed.com//cmp/_/job?jk=527f6e1ab8162f41&tk=1d3789e6b0mct000
posting - https://www.indeed.com//cmp/_/job?jk=62b9e0fd9cd1337a&tk=1d3789e6b0mct000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaeAqVDPpYphyIyfk7_KNcPtCwzxliBHgth61ePZK8zxYvigjMzhT0Wq_h0gtBMKQxuBLPylqeiuvOFPSRFi_PaGfJrmm1NIEqz-N9Wz7x8b5nqnxBEB3FFjRRmVOrGusHS9svCR5qeypJOSVsL7MePQH8PRdx7CUnSISd9KJiNBukXQ73IFiHsBMUGPvStRw39gPq3TaSyETWLmxY9fCBSzmnIziTsBC2o6blBP0nFZxPHvec9vqYO28Jxx1j7B68X9-QR_zvjrJT3HFuKiNWQ0fdSQ8TNWQC81wQqAOCBSmi_WYTxEmtx9ccT5MAAnYfI27AD3bPXkUO1xxjFET-A7Tc2IzLHJMen3paYW

page - https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start=108&clearPrefilter=1#cmp-menu-container
posting - https://www.indeed.com//cmp/_/job?jk=4f067edb256c3c93&tk=1d378al2f0mte000
posting - https://www.indeed.com//cmp/_/job?jk=4eea6480e1404fc5&tk=1d378al2f0mte000
posting - https://www.indeed.com//cmp/_/job?jk=80a407e7f1251cb1&tk=1d378al2f0mte000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXadQg5zIlQ-vocsrPoqU6Dl91Bhy-1EH5LROaaKfgK3zvQfEQl6YAvFiKJBgiFvHzmHeGQIUrZP7bi4pWHQMX7Y8fQ7pfR0YaqbB-DhuJ0w31giO9qt_alzcBpF4OcIxmyLkuXI2OT-XA9jZD3OjoNNmfUNTkmMBCEE_bbMJNnlHAyxbihy6lDkO8CNWriKQpbKn2JEp42U2h2tePQb5UE4YJY58bIwkQW48nhd5hCrxXppItYv1jHFX7doMQMPZHQTD7dKt6UP1mUuNkwkgA6mqaWA90bxxpSwCKyXQWkjIL5s5ehM6faZp9EofNb5-YyLUOR_LAOU-TspmziZ_MLPQppXIuxqvm295PgBOslbKLMeOS7DwLHzFVnV3TnVZwdNoeiWmwlqUMECon-9LZalWLvoW9y9hqYy76cfersqvvNzcrI8tu4pUHT21PykEGT_AaM886RodC6m3J42F1v7V2U2A38ckCYTIGuDtZUsUEmk47MCguL1SW3jhjjL7uC8
posting - h

...

posting - https://www.indeed.com//cmp/_/job?jk=f7a02963850d3a1f&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=0beb3b39910ed47a&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=952b65f991928ddd&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=337c6e6a7f7c96e2&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=21a789253e7b391d&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=e4f66d21a4c868da&tk=1d37a893l0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=14c7be66f316d13e&tk=1d37a893l0mtv000
page - https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start=792&clearPrefilter=1#cmp-menu-container
posting - https://www.indeed.com//cmp/_/job?jk=74a9f834f7438367&tk=1d37a8v5bakcg800
posting - https://www.indeed.com//cmp/_/job?jk=c65e9941ca896b17&tk=1d37a8v5bakcg800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaeNJQG_qUmKv5DOAAzCpXA

posting - https://www.indeed.com//cmp/_/job?jk=995da6b5d4ec684c&tk=1d37aii2n0mct000
posting - https://www.indeed.com//cmp/_/job?jk=a013e482d92fdc39&tk=1d37aii2n0mct000
posting - https://www.indeed.com//cmp/_/job?jk=7b91a95378f8d45d&tk=1d37aii2n0mct000
page - https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start=840&clearPrefilter=1#cmp-menu-container
posting - https://www.indeed.com//cmp/_/job?jk=bb473721693ea601&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=389bb090a7ae4b60&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=f52b74a6a6bda505&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=b75b970782d3c05d&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=e58c6f5bbc40c2c3&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=a0132f3943073aed&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk=8fa1833d2eb89fd1&tk=1d37aj4q2akcg800
posting - https://www.indeed.com//cmp/_/job?jk

posting - https://www.indeed.com//cmp/_/job?jk=a758ce9788d7bb28&tk=1d37al6ae0mtv000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6vxZlsYClqA2a8pSwxaDhLr79CqBG41ERir0PgmtH4LiZtm5HQhKjQkzcPmipSNpyC5tiagPlIXNQuHMEswaom70CrhxJC8DLBfD2B0aTheenX2ELwAPRO_Xm0NmDjUC7P_wq-CZgoAfuDxjKO2GNewMs5sXTMOgSszKyDg6_4QoXw_ZJcl-2W3SA4LIbM3Bu2k0TiV6O8R6RBrRhM5YLe9rZW4WkNab851Pouk6zQw5W-a6BoJP-IvZbmCCPnGno4Q1MwV6tVW2aQ-QCp4nLE2CKy4oErhb0A8nNLBQjqLYdEoCg906qV6U7i1f3TFs9_iQN_5UHl14wyY1UpK9A69o-lmCHSywBfoyU0oCnmMD46KzKTw3epSCd3tbB4ewF_07SEjY1EuX-riejpS8RI5_DK2A-omKppLVwr87Trl7sTHtcJkqmosX46LauB2PvdT_s-nLNi86USF6T0dlyeIBCF_7cGyUGa32QC56Ch26kpv-RTZhSToMhUeXjTRKLAsFWE06VYFvV34uBsb35Es
posting - https://www.indeed.com//cmp/_/job?jk=ae87a838e10fc87b&tk=1d37al6ae0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=9e0ad32ca2feb11d&tk=1d37al6ae0mtv000
posting - https://www.indeed.com//cmp/_/job?jk=b0691a3b90ded1ee&tk=1d37al6ae0mtv000
posting - https://www.indeed.com//cmp/_/j

posting - https://www.indeed.com//cmp/_/job?jk=a232e03fd68e46d9&tk=1d37an2l00kbu000
posting - https://www.indeed.com//cmp/_/job?jk=c114656025db58fe&tk=1d37an2l00kbu000
posting - https://www.indeed.com//cmp/_/job?jk=42c899c29a4b6509&tk=1d37an2l00kbu000&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXafe78Ip-nzSMqJQ_Ab9vZM2VQGvr10VF0k5It8jBWQ4z3-ZD4APoyVsGU8Wta2dhaOGDt2o7mC3yhBXQ8iDs4YuOnWENdQC6rKt75VxIvMNql0tVubm9fhEXlGNr7gC8ed8KQIel_Wp25O1zmFeA-yh6IXVUnmt_QCF2tO8Ru1-Zo8uvVV5epUdFXFeJdHuyZxxDvRImGTU9VZAV9EBWk-DDysZ7avbHX8-6GhWqhPdayWjYjR9zKNv6ZCOIMMxrqG-qpbstC-KLaxgEO2iF5mCH20BOxJfQoitdU8RyATApmfXVR_98LFD7tn57hjo-YRPhi_X5YtIWkdpuMho1Mw9V9ysCEEZhFKp9seR-ViLhyBrmTJUSUsVfQzBZ2NXJXXPuiHNpPP1nx_H4CQKqvy04rF46W_uPn0HzAT3nQ3EwgqQy7cgg_Zvabq94_Y2MafvCGcE2w0ypwSsnO-l8CibyoTYrYyPrXQf3YiFljAW16psGO80NJKnvCN70gz7LYQ
posting - https://www.indeed.com//cmp/_/job?jk=6f6bec976099c996&tk=1d37an2l00kbu000
posting - https://www.indeed.com//cmp/_/j

posting - https://www.indeed.com//cmp/_/job?jk=04e90aa9b45b02ce&tk=1d37ap8bhak8i800
posting - https://www.indeed.com//cmp/_/job?jk=8c183272e62d9c7a&tk=1d37ap8bhak8i800
posting - https://www.indeed.com//cmp/_/job?jk=0065130749f992d5&tk=1d37ap8bhak8i800
posting - https://www.indeed.com//cmp/_/job?jk=84b7e15020c5626f&tk=1d37ap8bhak8i800
posting - https://www.indeed.com//cmp/_/job?jk=c8d47b4f51159348&tk=1d37ap8bhak8i800&rd=n5r6n1S2hixwO8gpEj_fuBcDY5Ws-Xs_yuMBKiKofo91uS6Q69ktC-y-BiMHNS3ifTU3X4Uxq_eODsJbhaKG6pWY1xKe9vZtztShvwGdXaerH6qUBDSsRS62g--BFrzKTf66fSBWb5EdrI20w-VHoFk3ZwjMK78pjTg9mbKes-o2SZk7fN-Ocuj2Gg1kO8jkxMRPUv011FPYkSSFMEm7J0yhygSeW-W84RaRh6S8cV75h4Bz_9ud5G0PbmzUWiR_1JSp9fHHVxeSNnHJ-Xp7zr37TcA0lVPOi-uRzn6YG12pEc9A1UiLTbaEKKeyep1n0bjiMShRyZePi_o0w3ox-5CCgAySM5MxZyIWM_OFuwvMOSn2maTG_AXp2KZpBA75cftYHo7HGDyzbvR3fxrh7Y-BUoC9PN6g7Jz_mEB_Y6RKAjVIGq9k0ogmnTqE3fgwDgTncOP0YM5ruEaw8yHKiNAMN2ocHptO8gN5XPJrB78hyJywImgYXlhUesYKBjwlcvnvHG0hm3tvIoFYOCTt-xi9ToAp5or2FmPelMUFChiNVmLpVh1h-bZuUv9KEUd8b